In [1]:
import numpy as np
import cv2
from PIL import Image

import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [2]:
input_shape = (64, 64, 3)
target_size= (64,64)
num_classes = 36
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 
           'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 'o', 'p', 'q', 'r', 
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
#model_01.h5
model = Sequential([ 
    keras.Input(shape=input_shape),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Dropout(0.2),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Dropout(0.2),
    layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(num_classes, activation="softmax"),
])

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.load_weights('TrainedModels/model_01_64x64_b32.h5')

In [5]:
def predict_image_Cam(image, model):
    img_ = Image.fromarray(image)
    img_ = img_.resize(target_size)
    img_array = np.array(img_, dtype = float)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255
    prediction = model.predict(img_processed)
    return np.argmax(prediction)

In [6]:
def drawHandKeyPoints(handsValue, results, image):
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)

In [7]:
def handPrediction(image, handsValue):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = handsValue.process(image)
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    image = cv2.blur(image, (3,3))
    drawHandKeyPoints(handsValue, results, image)
    return image

In [8]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while True: 
        ret, frame = cap.read()
        
        image = handPrediction(frame, hands)
        index = predict_image_Cam(image, model)
        cv2.putText(image,'Prediction: ' + classes[index], (50, 50), font,
                    1, (0, 255, 100),  2, cv2.LINE_4)

        cv2.imshow('Sign Language Detection', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()